# Wireless Power Transfer System Design Methodolgy

## Introduction

The aim of this document is to come up with a system-level design guidelines and equations for a wireless power transfer system, operating in far-field mode. Such a system can be sized to wirelessly provide DC power to various consumers, such as sensor nodes, wearable electronics, drones, robots, etc. 

In the first instance, the system is assumed to be a point-to-point one, with wireless communication not taken into account. However, it should be fairly straighforward to transition to a multipoint-to-point system where several charging stations serve one end device, and to add communication capability using existing commercial solutions.

The outputs of this methodology are the following:
- Number of elements for the Tx antenna array $N_{Tx}$ and its respective gain $G_{Tx}$ 
- Number of elements for the Rx antenna array $N_{Rx}$ and its respective gain $G_{Rx}$
- The required Tx power $P_{Tx}$ before the antenna port


## Initial Assumptions

The main assumption and design goal is that system will be operating in the far-field, i.e. 

\begin{equation}
R \geq \frac{2D^2}{\lambda}
\end{equation}

where $R$ is the distance between the transmitter antenna (Tx) and receiver antenna (Rx), $\lambda$ is the wavelength corresponding to the centre frequency, and $D$ is the size of the aperture of the Tx antenna. In the case of non-square planar arrays, the larger dimension is used.

Following this, it is assumed that the designer of the system specifies the DC power required, $P_{DC}$, as well as the RF-DC conversion efficiency $\eta_{RF-DC}$. This efficiency is itself dependent on the frequency of operation, however it is a parameter that can be tweaked.

The method also assumes that both the distance $R$ and the wavelength $\lambda$ are specified, as the equations that have been derived are dependent on both. Initially, the results for $N_{Tx}$ and $N_{Rx}$ will be the arithmetically calculated ones, without rounding to the nearest number that can be expressed as $2^i$. Finally, the inter-element spacing in both $x$ and $y$ directions of the array is assumed to be $\dfrac{\lambda}{2}$.

In future iterations of this algorithm, it will be possible to have either $R$ or $\lambda$ as a free parameter, but other constraints will have to be introduced. Additionally, realistic array configurations will be considered.

## Algorithm Outline

1. Specify desired distance $R$, desired frequency of operation $f$, DC power required $P_{DC}$, and RF-DC conversion efficiency $\eta_{RF-DC}$.  
2. Calculate wavelength $\lambda$ and required input RF power after the Rx antenna $P_{RF}$:

\begin{align}
\lambda &= \frac{f}{c} \\
P_{RF} &= \frac{P_{DC}}{\eta_{RF-DC}}
\end{align}  

3. Assume/Measure values for losses in Tx and Rx feed networks, passive components, etc., $L_{Rx}$ and $L_{Tx}$. These can include return and insertion losses as well.  
4. Assume/Measure values for atmospheric losses, link margin, rain attenuation. Use $L_{Other}$ to represent these.
5. Calculate the Free Space Losses (FSL) from the Friis transmission equation:

\begin{equation}
FSL = 20\log_{10}\frac{4\pi d}{\lambda}
\end{equation}  

6. Calculate the following, giving the value for combined Tx and Rx antenna gain, as well as required Tx power $P_{Tx}$:

\begin{equation}
P_{Tx} + G_{Tx} + G_{Rx} = P_{RF} + FSL + L_{Rx} + L_{Tx} + L_{Other}
\end{equation}  

7. Calculate maximum number of array elements along one side of a square antenna array $N_{Tx}$:

\begin{align}
D &= N_{Tx} \cdot \frac{\lambda}{2} \\
N_{Tx} &\leq \sqrt{\frac{2R}{\lambda}}
\end{align}

8. Calculate Tx array gain $G_{Tx}$ and spot diameter $d$ at distance $R$ from the array:

\begin{align}
G_{Tx} &= 10\log_{10}N_{Tx}^2 \\
d &= 2R\tan{\frac{60}{N_{Tx}}}
\end{align}

9. Calculate the maximum size of a square Rx array that is fully illuminated by the main beam of the Tx array at distance $R$, and its corresponding gain $G_{Rx}$:

\begin{align}
A &= \frac{N_{Rx}^2\lambda^2}{4} \\
A &= \frac{d^2}{2} \\
N_{Rx} &= \frac{d\sqrt{2}}{\lambda} \\
G_{Rx} &= 10\log_{10}N_{Rx}^2
\end{align}

10. Determine required Tx power $P_{Tx}$, taking into account the gain of the individual array elements:

\begin{equation}
P_{Tx} = P_{RF} + FSL + L_{Rx} + L_{Tx} + L_{Other} - G_{Tx} - G_{Tx}^{CELL} - G_{Rx} - G_{Rx}^{CELL}
\end{equation}

## Example Design

### Design inputs:  
$R = 5 m = 500 cm$  
$f = 5.8 GHz \implies \lambda \approx 5.172 cm$  
$P_{DC} = 15 mW$  
$\eta_{RF-DC} = 0.6$

### Design calculations:

$P_{RF} = \frac{15}{0.6} = 25 mW \implies P_{RF} = 14 dBm$

Assume $L_{Rx} = L_{Tx} = 1.5 dB$ and $L_{Other} = 10 dB$  

$FSL = 20\log_{10}\frac{4\pi 500}{5.172} =  61.69 dB \approx 62 dB$  

$P_{Tx} + G_{Tx} + G_{Rx} = 14 + 3 + 10 + 62 = 89 dBm$

$N_{Tx} = \sqrt{\frac{2\cdot500}{5.172}} = 13.91 \approx 13$  

$G_{Tx} = 10\log_{10}13^2 = 22.28 dBi$  

$d = 2\cdot500\tan{\frac{60}{13}} = 80.73 cm$  

$N_{Rx} = \frac{80.73\sqrt{2}}{5.172} = 22.075 \approx 22$  

$G_{Rx} = 10\log_{10}22^2 = 26.84 dBi$

Assume $G_{Tx}^{CELL} = G_{Rx}^{CELL} = 6 dBi$  

$P_{Tx} = 89 - 22.28 - 26.84 - 6 - 6 = 27.88 dBm \approx 620 mW$

## Python implementation of the above algorithm

In [28]:
import numpy as np
from scipy.constants import speed_of_light

distance = 5  # distance in metres
freq = 5.8  # frequency in GHz
p_dc = 15  # required DC power in mW
n = 0.6  # assumed RF-DC conversion efficiency

# assumed additional losses and gains, all in dB
l_rx = 1.5
l_tx = 1.5
l_other = 10
g_cell_tx = 6
g_cell_rx = 6

# calculate required input RF power in mW and dBm
p_rf = p_dc / n
p_rf_db = 10 * np.log10(p_rf)

# calculate free space loss
wavelength = speed_of_light / (freq * 1e9)
fsl = 20 * np.log10(4 * np.pi * distance / wavelength)

total_losses = fsl + l_rx + l_tx + l_other

# calculate maximum number of elements in Tx array
n_tx = np.sqrt(2 * distance / wavelength)
n_tx = np.trunc(n_tx)  # always round down
g_tx = 10 * np.log10(n_tx ** 2)

# calculate spot diameter and size of largest inscribed square array, along with its gain
spot_diameter = 2 * distance * np.tan(np.radians(60 / n_tx))
n_rx = spot_diameter * np.sqrt(2) / wavelength
n_rx = np.trunc(n_rx)
g_rx = 10 * np.log10(n_rx ** 2)

# calculate required transmit RF power
p_tx_db = p_rf_db + total_losses - g_rx - g_tx - g_cell_rx - g_cell_tx
p_tx = 10 ** (p_tx_db / 10.0)

# print out results
print("Required DC power: {0:0.2f} mW / {1:0.2f} dBm".format(p_rf, p_rf_db))
print("Free space losses: {0:0.2f} dB".format(fsl))
print("Total losses: {0:0.2f} dB".format(total_losses))
print("Maximum number of elements in a square array in one direction: {0}".format(n_tx))
print("Corresponding array gain: {0:0.2f} dBi".format(g_tx))
print("Spot diameter at {0} m: {1:0.2f} cm".format(distance, spot_diameter * 1e2))
print("Maximum number of elements in an inscribed square array, in one direction: {0}".format(n_rx))
print("Corresponding arragy gain: {0:0.2f} dBi".format(g_rx))
print("Required RF power: {0:0.2f} mW / {1:0.2f} dBm".format(p_tx, p_tx_db))

Required DC power: 25.00 mW / 13.98 dBm
Free space losses: 61.70 dB
Total losses: 74.70 dB
Maximum number of elements in a square array in one direction: 13.0
Corresponding array gain: 22.28 dBi
Spot diameter at 5 m: 80.73 cm
Maximum number of elements in an inscribed square array, in one direction: 22.0
Corresponding arragy gain: 26.85 dBi
Required RF power: 568.57 mW / 27.55 dBm
